In [6]:
import os, sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [75]:
import classification.data_loading as cdl
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import LeaveOneOut
from scipy.spatial.distance import mahalanobis
import numpy as np
from e_nose.measurements import DataType
from numpy.random import randint

In [8]:
measurements = cdl.get_measurements_from_dir("../data")
print("length measurements: ", len(measurements))

Reading file ../data/data_raisin_coffee_powder_orange_juice_red_wine_isopropanol_wodka_10_loops_for_5.0_min_referenceTime_5.0_min_2020-3-3_9_53_cleaned.csv
using sensor 5 specification
Reading file ../data/data_raisin_coffee_powder_orange_juice_red_wine_isopropanol_wodka_20_loops_for_5.0_min_referenceTime_5.0_min_2020-3-4_18_0_cleaned.csv
using sensor 5 specification
Reading file ../data/data_raisin_coffee_powder_orange_juice_red_wine_isopropanol_wodka_10_loops_for_5.0_min_referenceTime_5.0_min_2020-3-2_18_14_cleaned.csv
using sensor 5 specification
Reading file ../data/data_raisin_coffee_powder_orange_juice_red_wine_isopropanol_wodka_20_loops_for_5.0_min_referenceTime_5.0_min_2020-3-3_20_23_cleaned.csv
using sensor 5 specification
Reading file ../data/data_coffee_powder_orange_juice_isopropanol_raisin_red_Wine_wodka_10_loops_for_5.0_min_referenceTime_30.0_min_2020-2-28_7_57_cleaned.csv
using sensor 5 specification
Read 5 files
length measurements:  317


In [9]:
training, test = cdl.train_test_split(measurements)
training_X = []
training_Y = []
test_X = []
test_Y = []

In [15]:
for i, measurement in enumerate(test[100::]):
    for x, tmp in enumerate(measurement.get_data_as(DataType.HIGH_PASS)):
        test_X.append(tmp)
        test_Y.append(measurement.label)

for i, measurement in enumerate(training[100::]):
    for x, tmp in enumerate(measurement.get_data_as(DataType.HIGH_PASS)):
        training_X.append(tmp)
        training_Y.append(measurement.label)

In [ ]:
#cov_X = np.cov(training_X)

In [7]:
#def mahalanobis(a, b):
#    return mahalanobis(a, b, cov_X)

In [11]:
def euk_distance(x,y):
    return numpy.linalg.norm(x-y)

In [41]:
knn_models = None
highest_acc = 0
for i in range(1, 21):
    fitting = KNeighborsClassifier(i,'distance',metric='minkowski',p=2,algorithm='auto')#euclidian distance
    fitting.fit(training_X, training_Y)
    score = fitting.score(test_X,test_Y)
    if(score > highest_acc):
        highest_acc=score
        knn_models = fitting
        
    print("accuracy for ", i, " neighbours: ",score)

accuracy for  1  neighbours:  0.5136363636363637
accuracy for  2  neighbours:  0.5136363636363637
accuracy for  3  neighbours:  0.5148760330578512
accuracy for  4  neighbours:  0.5161845730027548
accuracy for  5  neighbours:  0.5207300275482094
accuracy for  6  neighbours:  0.5243112947658403
accuracy for  7  neighbours:  0.5243801652892562
accuracy for  8  neighbours:  0.5263085399449036
accuracy for  9  neighbours:  0.5269972451790633
accuracy for  10  neighbours:  0.528374655647383
accuracy for  11  neighbours:  0.5274793388429752
accuracy for  12  neighbours:  0.528236914600551
accuracy for  13  neighbours:  0.5292699724517906
accuracy for  14  neighbours:  0.5301652892561983
accuracy for  15  neighbours:  0.5308539944903581
accuracy for  16  neighbours:  0.531404958677686
accuracy for  17  neighbours:  0.5317493112947659
accuracy for  18  neighbours:  0.5320936639118458
accuracy for  19  neighbours:  0.5331955922865014
accuracy for  20  neighbours:  0.5333333333333333


In [49]:
knn_regressor_models = None
highest_regressor_acc = 0
for i in range(1, 21):
    fitting = KNeighborsRegressor(i,'distance',metric='minkowski',p=2,algorithm='auto')#euclidian distance
    fitting.fit(training_X, training_Y)
    score = fitting.score(test_X,test_Y)
    if(score > highest_regressor_acc):
        highest_regressor_acc=score
        knn_regressor_models = fitting
        
    print("accuracy for ", i, " neighbours: ",score)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [42]:
from sklearn.externals import joblib
filename = 'knn_classifier.joblib.pkl'
_ = joblib.dump(knn_models, filename, compress=9)  

In [55]:
fitting: KNeighborsClassifier = joblib.load(filename)

In [86]:
for i in randint(0,5000,50):
    pred_per = fitting.predict_proba([test_X[i]])
    pred = fitting.predict([test_X[i]])
    print(i,': pred: ',pred,'real: ',test_Y[i],' per: ',pred_per)

225 : pred:  ['red_wine'] real:  orange_juice  per:  [[0. 0. 0. 0. 1. 0.]]
4192 : pred:  ['coffee_powder'] real:  coffee_powder  per:  [[0.49965715 0.         0.04920774 0.45113511 0.         0.        ]]
2819 : pred:  ['isopropanol'] real:  isopropanol  per:  [[0. 1. 0. 0. 0. 0.]]
2540 : pred:  ['coffee_powder'] real:  coffee_powder  per:  [[0.89680035 0.         0.10319965 0.         0.         0.        ]]
1722 : pred:  ['raisin'] real:  raisin  per:  [[0. 0. 0. 1. 0. 0.]]
4540 : pred:  ['isopropanol'] real:  isopropanol  per:  [[0. 1. 0. 0. 0. 0.]]
4399 : pred:  ['wodka'] real:  wodka  per:  [[0. 0. 0. 0. 0. 1.]]
336 : pred:  ['raisin'] real:  raisin  per:  [[0.20065915 0.         0.09597705 0.55404322 0.14932058 0.        ]]
13 : pred:  ['isopropanol'] real:  isopropanol  per:  [[0. 1. 0. 0. 0. 0.]]
4097 : pred:  ['raisin'] real:  coffee_powder  per:  [[0.04898076 0.         0.         0.95101924 0.         0.        ]]
1679 : pred:  ['raisin'] real:  raisin  per:  [[0. 0. 0. 1. 0